In [18]:
import requests

cookies = {

}

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:132.0) Gecko/20100101 Firefox/132.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    'Referer': 'https://habr.com/ru/search/?q=data+&target_type=posts&order=relevance',
    'habr-user-uuid': '62Y9wdJ0cFHAZOQG21LKrpqxQExPW1lk2q81x0AGJ/irGryxh/h2McwLyK7KArysBpjCsDirUexMuiR1PslIXQ',
    'x-app-version': '2.211.2',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'Connection': 'keep-alive',
    'Priority': 'u=0',
}

params = {
    'query': 'python',
    'order': 'relevance',
    'fl': 'ru',
    'hl': 'ru',
    'page': '1',
    'perPage': '20',
}

response = requests.get('https://habr.com/kek/v2/articles/', params=params, cookies=cookies, headers=headers)

In [19]:
response_json = response.json()

In [20]:
import json

with open('habr.json', 'w', encoding='utf-8') as file:
    json.dump(response_json, file, ensure_ascii=False, indent=4)


In [21]:
with open('habr.json', 'r', encoding='utf-8') as file:
    data = json.load(file)



# решение с парсингом страницы а не json

In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

LINK = "https://habr.com"

In [51]:
def get_data(querry, page):
    url = f"https://habr.com/ru/search/page{str(page)}/?q={querry}&target_type=posts&order=relevance"
    response = requests.get(url)
    return response.text



In [53]:
def get_info(soup):
    all_articles_habr = soup.find_all("article", class_="tm-articles-list__item")
    articles = []
    for article in all_articles_habr:
        title = article.find("h2").text
        author_raw = article.find("span", class_="tm-user-info__user")
        author = author_raw.find('a', class_='tm-user-info__username').text
        sub_link = author_raw.find('a', class_='tm-article-datetime-published_link')['href']
        date = author_raw.find('time')['datetime']
        articles.append({
            "title": title,
            "author": author,
            "link": LINK + sub_link,
            "date": date
        })
    return articles

In [54]:
soup = BeautifulSoup(get_data("python", 1), 'html.parser')

all_articles = get_info(soup)
len(all_articles)

20

In [56]:
df = pd.DataFrame(all_articles)
df.to_csv('habr_python.csv', index=False)
df.head()

,title,author,link,date
0,Курс «Python для инженеров». Старт 3 потока 31...,Hedgehog_art,https://habr.com/ru/companies/slurm/news/646825/,2022-01-20T15:37:16.000Z
1,Жаждущим автоматизации: открытый урок «ChatOps...,geniyoctober,https://habr.com/ru/companies/slurm/news/595093/,2021-12-13T06:00:03.000Z
2,Открытый урок «Пишем Custom Prometheus Exporte...,Hedgehog_art,https://habr.com/ru/companies/slurm/news/645485/,2022-01-13T15:35:30.000Z
3,Microsoft представила дополнение Python Editor...,denis-19,https://habr.com/ru/news/761862/,2023-09-19T04:42:15.000Z
4,55% Python-разработчиков используют Linux-окру...,maybe_elf,https://habr.com/ru/news/840224/,2024-09-02T07:27:25.000Z
